# Session 3:  Analysis Tools

<a id='trajanalysis'></a>

<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Creative Commons Licence" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" title='This work is licensed under a Creative Commons Attribution 4.0 International License.' align="right"/></a>

Authors: 

- Dr Micaela Matta - [@micaela-matta](https://github.com/micaela-matta)
- Dr Richard Gowers - [@richardjgowers](https://github.com/richardjgowers) 

This notebook is adapted from materials developed for the [2018 Workshop/Hackathon](https://github.com/MDAnalysis/WorkshopHackathon2018) and the [MDAnalysis User Guide](https://userguide.mdanalysis.org/stable/examples/analysis/custom_trajectory_analysis.html#Creating-an-analysis-from-a-function).

### Learning outcomes 

- Using built-in analysis methods that act on `positions` and iterating over a trajectory
- Using analysis methods that act on `Atomgroups`, such as radial distribution function, persistence length, root mean square deviation (RMSD)
- Creating your own analysis tools with MDAnalysis


#### Additional resources

 - During the workshop, feel free to ask questions at any time
 - For more on how to use MDAnalysis, see the [User Guide](https://userguide.mdanalysis.org/2.0.0-dev0/) and [documentation](https://docs.mdanalysis.org/2.0.0-dev0/)
 - Ask questions on the [GitHub Discussions forum](https://github.com/MDAnalysis/mdanalysis/discussions) or on [Discord](https://discord.gg/fXTSfDJyxE)
 - Report bugs on [GitHub](https://github.com/MDAnalysis/mdanalysis/issues?)


# Google Colab package installs

This installs the necessary packages for Google Colab. Please only run these if you are using Colab.

In [ ]:
# NBVAL_SKIP
!pip install condacolab
import condacolab
condacolab.install()

In [ ]:
# NBVAL_SKIP
import condacolab
condacolab.check()
!mamba install -c conda-forge mdanalysis mdanalysistests mdanalysisdata nglview rdkit

In [ ]:
# NBVAL_SKIP
# enable third party jupyter widgets
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
import warnings
warnings.filterwarnings("ignore") 

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import nglview as nv
import MDAnalysis as mda
from MDAnalysis.lib import distances 
from MDAnalysis.analysis import rdf
import MDAnalysisData as data



## 1. `radius_of_gyration` and end-to-end distance calculation for the whole trajectory
Let's go back to our PEG chain and calculate the `radius_of_gyration`:

In [ ]:
PEG_example = data.datasets.fetch_PEG_1chain()

In [ ]:
peg_u = mda.Universe(PEG_example['topology'], PEG_example['trajectory'])

In [ ]:
peg_u.atoms.residues[0]

In [ ]:
rog = []
Os = peg_u.select_atoms('type os')
for ts in peg_u.trajectory:
    rog.append(Os.radius_of_gyration())

#now let's plot:
plt.plot(rog)
plt.xlabel('frame')
plt.ylabel(r"R$_{g}$ ($\AA$)")
plt.show()


<div class="alert alert-success"> 

**Exercise 1:**

Calculate and plot the center of mass of PEG over the entire trajectory.

</div> 

## 2. Radial Distribution Function (RDF) calculation
<a id='rdf'></a>

The [radial distribution function](https://en.wikipedia.org/wiki/Radial_distribution_function) describes the probability of finding an atom/molecule within a certain distance of another. The MDAnalysis analysis class has a built-in `rdf` tool for this purpose. 

<img src="https://upload.wikimedia.org/wikipedia/commons/3/3b/Rdf_schematic.svg"  width="600" height="300">

In this case, we don't have to iterate over the trajectory because the function already does that for us.

We will demonstrate how to calculate the RDF between PEG oxygens and water oxygens. Let's first define our atomgroups:

In [ ]:
ow = peg_u.select_atoms('type OW')

op = peg_u.select_atoms('resname UNL and element O')

We use the `rdf.InterRDF` function to calculate the radial distribution function:

In [ ]:
OO_rdf = rdf.InterRDF(ow, op, range=(0,10))

We need to `.run()` the radial distribution function, and optionally select an interval of frames:

In [ ]:
RDF = OO_rdf.run()

This returns a `results` dictionary reporting the `bins` and `rdf` values:

In [ ]:
RDF.results['bins']

In [ ]:
RDF.results['rdf']

We can now plot the RDF between the oxygens of water and PEG:

In [ ]:
plt.plot(RDF.results['bins'], RDF.results['rdf'])
plt.hlines(1, 0, 10, linestyles='dashed', colors='orange')
plt.xlabel('r ($\AA$)')
plt.ylabel(r'O$_{W}$ - O$_{PEG}$ RDF')
plt.show()

## 3. Persistence length 

This is also a built-in method that return the persistence length calculated over the whole trajectory.

In analysing polymers, the persistence length is a measure of a chains stiffness.  The persistence length is the distance at which the direction of two points on a polymer chain becomes decorrelated.  High persistence lengths indicate that the polymer chain is rigid and doesn't change direction, low persistence lengths indicate that the polymer chain has little memory of its orientation.

The bond autocorrelation function $C(n)$ measures the average cosine of the angle between bond vector $\mathbf{a_i}$ and a bond vector $n$ bonds away. 

$$C(n) = \langle \cos\theta_{i, i+n} \rangle= \langle \mathbf{a_i} \cdot \mathbf{a_{i+n}} \rangle$$

This is then fitted to an exponential decay, where $l_B$ is the average bond length, and $l_P$ is the persistence length.


$$C(n) \approx \exp\left(-\frac{n l_B}{l_P}\right)$$


In [ ]:
from MDAnalysis.analysis.polymer import PersistenceLength

Select the backbone of the polymer. It's easy in this case since we only need to exclude hydrogens:

In [ ]:
backbone=peg_u.select_atoms('name C*')
backbone

It is important that the contents of the polymer `atomgroup` are in order. 
Selections done using `select_atoms` will always be sorted.
This can be checked by listing the `atomgroup`.

In [ ]:
list(backbone[:10])

Run the `PersistenceLength` function: 

In [ ]:
pl=PersistenceLength([backbone]).run()

In [ ]:
pl.results

We can plot the autocorrelation using `pl.results`.
The tool also returns the exponential decay fit for us, which yields the persistence length `lp`.
We can check the validity of the fit by plotting the results:

In [ ]:
plt.plot(pl.results['x'], pl.results['bond_autocorrelation'], 'o')
plt.plot(pl.results['x'], pl.results['fit'])
plt.show()

.. Or just look at the value of $l_{P}$:

In [ ]:
pl.results['lp']

## 4. Root mean square deviation (RMSD)

Here we calculate the RMSD of domains in adenylate kinase (AdK), a phosophotransferase enzyme, as it transitions from an open to closed structure.

In [ ]:
from MDAnalysis.tests.datafiles import PSF, DCD, CRD
from MDAnalysis.analysis import rms

import pandas as pd

The trajectory DCD samples a transition from a closed to an open conformation. AdK has three domains:

- CORE
- LID: an ATP-binding domain
- NMP: an AMP-binding domain

The LID and NMP domains move around the stable CORE as the enzyme transitions between the opened and closed conformations. One way to quantify this movement is by calculating the root mean square deviation (RMSD) of atomic positions.

In [ ]:
u = mda.Universe(PSF, DCD)  # closed AdK (PDB ID: 1AKE)
ref = mda.Universe(PSF, CRD)  # open AdK (PDB ID: 4AKE)

## Background: RMSD

The root mean square deviation (RMSD) of particle coordinates is one measure of distance, or dissimilarity, between molecular conformations. Each structure should have matching elementwise atoms $i$ in the same order, as the distance between them is calculated and summed for the final result. It is calculated between coordinate arrays $\mathbf{x}$ and $\mathbf{x}^{\text{ref}}$ according to the equation below:

$$ \text{RMSD}(\mathbf{x}, \mathbf{x}^{\text{ref}}) = \sqrt{\frac{1}{n} \sum_{i=1}^{n}{|\mathbf{x}_i-\mathbf{x}_i^{\text{ref}}|^2}} $$

As molecules can move around, the structure $\mathbf{x}$ is usually translated by a vector $\mathbf{t}$ and rotated by a matrix $\mathsf{R}$ to align with the reference $\mathbf{x}^{\text{ref}}$ such that the RMSD is minimised. The RMSD after this optimal superposition can be expressed as follows:

$$ \text{RMSD}(\mathbf{x}, \mathbf{x}^{\text{ref}}) = \min_{\mathsf{R}, \mathbf{t}} %
  \sqrt{\frac{1}{N} \sum_{i=1}^{N} \left[ %
      (\mathsf{R}\cdot\mathbf{x}_{i}(t) + \mathbf{t}) - \mathbf{x}_{i}^{\text{ref}} \right]^{2}}$$

The RMSD between one reference state and a trajectory of structures is often calculated as a way to measure the dissimilarity of the trajectory conformational ensemble to the reference. This reference is frequently the first frame of the trajectory (the default in MDAnalysis), in which case it can provide insight into the overall movement from the initial starting point. W

Typically not all coordinates in a structures are included in an RMSD analysis. With proteins, the fluctuation of the residue side-chains is not representative of overall conformational change. Therefore when RMSD analyses are performed to investigate large-scale movements in proteins, the atoms are usually restricted only to the backbone atoms (forming the amide-bond chain) or the alpha-carbon atoms. 

MDAnalysis provides functions and classes to calculate the RMSD between coordinate arrays, and `Universes` or `AtomGroups`.  The contribution of each particle $i$ to the final RMSD value can also be weighted.


### RMSD between two sets of coordinates

The MDAnalysis.analysis.rms.rmsd function returns the root mean square deviation (in Angstrom) between two sets of coordinates. Here, we calculate the RMSD between the backbone atoms of the open and closed conformations of AdK. 

In [ ]:
rms.rmsd(u.select_atoms('backbone').positions,  # coordinates to align
         ref.select_atoms('backbone').positions,  # reference coordinates
         center=True,  # subtract the center of geometry
         superposition=True)  # superimpose coordinates

### RMSD of a Universe with multiple selections

It is more efficient to use the `MDAnalysis.analysis.rms.RMSD` class to calculate the RMSD of an entire trajectory to a single reference point, than to use the the `MDAnalysis.analysis.rms.rmsd` function.

The `rms.RMSD` class first performs a rotational and translational alignment of the target trajectory to the reference universe at `ref_frame`, using the atoms in select to determine the transformation. The RMSD of the select selection is calculated. Then, without further alignment, the RMSD of each group in `groupselections` is calculated.

In [ ]:
CORE = 'backbone and (resid 1-29 or resid 60-121 or resid 160-214)'
LID = 'backbone and resid 122-159'
NMP = 'backbone and resid 30-59'

In [ ]:
R = rms.RMSD(u,  # universe to align
             u,  # reference universe or atomgroup
             select='backbone',  # group to superimpose and calculate RMSD
             groupselections=[CORE, LID, NMP],  # groups for RMSD
             ref_frame=0)  # frame index of the reference
R.run()

The data is saved as usual in the `results` object:

In [ ]:
R.results.rmsd

### Plotting the data

We can easily plot this data using the common data analysis package `pandas`. We turn the `R.rmsd` array into a `DataFrame` and label each column below.


In [ ]:
df = pd.DataFrame(R.results.rmsd,
                  columns=['Frame', 'Time (ns)',
                           'Backbone', 'CORE',
                           'LID', 'NMP'])

df

In [ ]:
ax = df.plot(x='Frame', y=['Backbone', 'CORE', 'LID', 'NMP'],
             kind='line')
ax.set_ylabel(r'RMSD ($\AA$)')

<div class="alert alert-info"> <b> Reminder: </b> 


Some analysis tools (`radius_of_gyration`, `center_of_mass` etc) act on `positions` (which are properties of a single `timestep`). To calculate the property of interest for each frame, we need to iterate over the whole trajectory. 

Others (`RDF`, `rms.RMSD` etc) act on `AtomGroups`, and can iterate over the whole trajectory for us.

</div>

## 5. Writing your own analysis method

We will now demonstrate how to create your own analysis methods.

This can generally be done in three ways, from least to most flexible:

 1. [Running the analysis directly from a function](#Creating-an-analysis-from-a-function)
 
 2. [Turning a function into a class](#Transforming-a-function-into-a-class)
 
 3. [Writing your own class](#Creating-your-own-class)

Note: the building blocks and methods shown in this section are only suitable for analyses that involve iterating over the trajectory once.

**If you implement your own analysis method, please consider [contributing it to the MDAnalysis codebase!](https://www.mdanalysis.org/UserGuide/contributing.html)**


In [ ]:
from MDAnalysis.tests.datafiles import PSF, DCD, DCD2
from MDAnalysis.analysis.base import (AnalysisBase,
                                      AnalysisFromFunction,
                                      analysis_class)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Radius of gyration

Let's start off by defining a standalone analysis function.

The radius of gyration of a structure measures how compact it is. In [GROMACS](http://manual.gromacs.org/documentation/2019-rc1/reference-manual/analysis/radius-of-gyration.html), it is calculated as follows: 

$$ R_g = \sqrt{\frac{\sum_i m_i \mathbf{r}_i^2}{\sum_i m_i}}$$

where $m_i$ is the mass of atom $i$ and $\mathbf{r}_i$ is the position of atom $i$, relative to the center-of-mass of the selection.

The radius of gyration around each axis can also be determined separately. For example, the radius of gyration around the x-axis:

$$ R_{i, x} = \sqrt{\frac{\sum_i m_i [r_{i, y}^2 + r_{i, z}^2]}{\sum_i m_i}}$$

Below, we define a function that takes an AtomGroup and calculates the radii of gyration. We could write this function to only need the AtomGroup. However, we also add in a `masses` argument and a `total_mass` keyword to avoid recomputing the mass and total mass for each frame.

In [ ]:
def radgyr(atomgroup, masses, total_mass=None):
    # coordinates change for each frame
    coordinates = atomgroup.positions
    center_of_mass = atomgroup.center_of_mass()
    
    # get squared distance from center
    ri_sq = (coordinates-center_of_mass)**2
    # sum the unweighted positions
    sq = np.sum(ri_sq, axis=1)
    sq_x = np.sum(ri_sq[:,[1,2]], axis=1) # sum over y and z
    sq_y = np.sum(ri_sq[:,[0,2]], axis=1) # sum over x and z
    sq_z = np.sum(ri_sq[:,[0,1]], axis=1) # sum over x and y
    
    # make into array
    sq_rs = np.array([sq, sq_x, sq_y, sq_z])
    
    # weight positions
    rog_sq = np.sum(masses*sq_rs, axis=1)/total_mass
    # square root and return
    return np.sqrt(rog_sq)

### Loading files

The test files we will be working with here feature adenylate kinase (AdK), a phosophotransferase enzyme. (<a data-cite="beckstein_zipping_2009" href="https://doi.org/10.1016/j.jmb.2009.09.009">Beckstein *et al.*, 2009</a>)

In [ ]:
u = mda.Universe(PSF, DCD)
protein = u.select_atoms('protein')

u2 = mda.Universe(PSF, DCD2)

### Creating an analysis from a function

`MDAnalysis.analysis.base.AnalysisFromFunction` can create an analysis from a function that works on AtomGroups. It requires the function itself, the trajectory to operate on, and then the arguments / keyword arguments necessary for the function.

In [ ]:
rog = AnalysisFromFunction(radgyr, u.trajectory, 
                           protein, protein.masses, 
                           total_mass=np.sum(protein.masses))
rog.run()

Running the analysis iterates over the trajectory. The output is saved in `rog.results.timeseries`, which has the same number of rows, as frames in the trajectory. You can access the results both at `rog.results.timeseries` and `rog.results['timeseries']`:

In [ ]:
rog.results['timeseries'].shape

gives the same outputs as:

In [ ]:
rog.results.timeseries.shape

In [ ]:
labels = ['all', 'x-axis', 'y-axis', 'z-axis']
for col, label in zip(rog.results['timeseries'].T, labels):
    plt.plot(col, label=label)
plt.legend()
plt.ylabel('Radius of gyration (Å)')
plt.xlabel('Frame')

You can also re-run the analysis with different frame selections. 

Below, we start from the 10th frame and take every 8th frame until the 80th. Note that the slice includes the `start` frame, but does not include the `stop` frame index (much like the actual `range()` function).

In [ ]:
rog_10 = AnalysisFromFunction(radgyr, u.trajectory, 
                              protein, protein.masses, 
                              total_mass=np.sum(protein.masses))

rog_10.run(start=10, stop=80, step=7)
rog_10.results['timeseries'].shape

In [ ]:
for col, label in zip(rog_10.results['timeseries'].T, labels):
    plt.plot(col, label=label)
plt.legend()
plt.ylabel('Radius of gyration (Å)')
plt.xlabel('Frame')

### Transforming a function into a class

While the `AnalysisFromFunction` is convenient for quick analyses, you may want to turn your function into a class that can be applied to many different trajectories, much like other MDAnalysis analyses.

You can apply `analysis_class` to any function that you can run with `AnalysisFromFunction` to get a class.

In [ ]:
RadiusOfGyration = analysis_class(radgyr)

To run the analysis, pass exactly the same arguments as you would for `AnalysisFromFunction`.

In [ ]:
rog_u1 = RadiusOfGyration(u.trajectory, protein, 
                          protein.masses,
                          total_mass=np.sum(protein.masses))
rog_u1.run()

As with `AnalysisFromFunction`, the results are in `results`.

In [ ]:
for col, label in zip(rog_u1.results['timeseries'].T, labels):
    plt.plot(col, label=label)
plt.legend()
plt.ylabel('Radius of gyration (Å)')
plt.xlabel('Frame')
plt.show()

## Contributing to MDAnalysis

If you think that you will want to reuse your new analysis, or that others might find it helpful, please consider [contributing it to the MDAnalysis codebase.](https://www.mdanalysis.org/UserGuide/contributing.html) Making your code open-source can have many benefits; others may notice an unexpected bug or suggest ways to optimise your code. If you write your analysis for a specific publication, please let us know; we will ask those who use your code to cite your reference in published work.